In [5]:
import argparse
import time
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler

import data
import model

from utils import batchify, get_batch, repackage_hidden
#import main

In [2]:

def evaluate(data_source, batch_size=10):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    if args.model == 'QRNN': model.reset()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(batch_size)
    for i in range(0, data_source.size(0) - 1, args.bptt):
        #print('eval:',i)
        #print('before get_batch,  max_memory_reserved():',torch.cuda.max_memory_reserved()/1024/1024)
        data, targets = get_batch(data_source, i, args, evaluation=True)
        #print('after get_batch,  max_memory_reserved():',torch.cuda.max_memory_reserved()/1024/1024)
        output, hidden = model(data, hidden)
        total_loss += len(data) * criterion(model.decoder.weight, model.decoder.bias, output, targets).data
        hidden = repackage_hidden(hidden)
    return total_loss.item() / len(data_source)

In [8]:
def model_load(fn):
    global model, criterion, optimizer
    #if args.philly:
        #fn = os.path.join(os.environ['PT_OUTPUT_DIR'], fn)
    #fn = '1591009087197056.pt'
    with open(fn, 'rb') as f:
        model, criterion, optimizer = torch.load(f)


In [29]:
#print('Loading cached dataset...')
#corpus = torch.load('corpus.148650ff682fa3f76e78c18d7d6d5bd6.data')

print('Producing dataset...')
corpus = data.Corpus('data/penn2/')
#torch.save(corpus, fn)

Producing dataset...


In [30]:
test_batch_size = 1
#args = {'cuda':True}
class arg1:
    cuda = True
    model  = 'LSTM'
    bptt = 70
args = arg1()
test_data = batchify(corpus.test, test_batch_size, args)

In [31]:
print(test_data.shape)

torch.Size([213, 1])


In [32]:
# Load the best saved model.
model_load('1591009087197056.pt')

# Run on test data.
test_loss = evaluate(test_data, test_batch_size)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f} | test bpc {:8.3f}'.format(
    test_loss, math.exp(test_loss), test_loss / math.log(2)))
print('=' * 89)

| End of training | test loss  7.57 | test ppl  1946.00 | test bpc   10.926
